In [ ]:
from pathlib import Path
import glob as gb
import re

In [ ]:
ROOT_DIR = Path('.').absolute().parent.parent
ROOT_DIR

# Initialize vertices directly

In [ ]:
cpp_files = gb.glob(str(ROOT_DIR / 'src' / '**' / '*.cpp'), recursive=True)

In [ ]:
#cpp_files = [ '/Users/julien/Software/Others/OpenStudio/src/model/test/SubSurface_GTest.cpp',]

In [ ]:
RE_VERTICES = re.compile(r'^(\s*)(?:std::vector<Point3d>|Point3dVector) (\w+);')
RE_VERTICES_CLEAR = re.compile(r'^(\s*)(\w+).clear\(\);')
RE_PUSHBACK = re.compile(r'\w+.push_back\(Point3d\(\s*(-?\d+(?:\.\d+)?),\s*(-?\d+(?:\.\d+)?),\s*(-?\d+(?:\.\d+)?)\)\);')
RE_PUSHBACK = re.compile(r'\w+.push_back\(Point3d\(\s*([^,]+),\s*([^,]+),\s*([^,]+)\)\);')

In [ ]:
for cpp_file in cpp_files:
    with open(cpp_file, 'r') as f:
        content = f.read()
    lines = content.splitlines()
    done_lines = []
    new_lines = []
    anyDone = False
    for i, line in enumerate(lines):
        if i in done_lines:
            continue
        done_lines.append(i)
        if (m11 := RE_VERTICES.search(line)) or (m12 := RE_VERTICES_CLEAR.search(line)):
            if m11:
                spacing = m11.groups()[0]
            else:
                spacing = m12.groups()[0]
            print(i, line)
            j = i+1
            pts = []
            while (m2 := RE_PUSHBACK.search(lines[j])):
                anyDone = True
                pts.append(m2.groups())
                done_lines.append(j)
                j += 1
            if pts:
                if m11:
                    new_lines.append(line.replace(';', ' {'))
                else:
                    new_lines.append(line.replace('.clear();', ' = {'))
                for k, pt in enumerate(pts):
                    new_line = spacing + '  {{{}, {}, {}}}'.format(*pt)
                    # if k != (len(pts) - 1):
                    new_line += ','
                    new_lines.append(new_line)

                new_lines.append(spacing + '};')
            else:
                for k in range(i, j):
                    new_lines.append(lines[i])
        else:
            new_lines.append(line)
        
    if anyDone:
        with open(cpp_file, 'w') as f:
            f.write("\n".join(new_lines) + '\n')
            